## Features from Option data
   * large flows

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
from cdcqr.data.dataloader import data_loader
from cdcqr.data.deribit.data_utils import DeribitUtils, quoto_and_trade_analysis
from cdcqr.common.config import LOCAL_DATA_DIR
import os

### Data commom

In [ ]:
exchange = 'deribit'

In [ ]:
date_range = pd.date_range(datetime(2021,1,1), datetime(2021,10,31))

In [ ]:
data_type= 'trades'
date = date_range[0]
symbol='OPTIONS'

In [ ]:
df_opt_trade = data_loader(exchange, date,"trades", symbol).pipe(DeribitUtils.parse_time_col).pipe(DeribitUtils.parse_optSymbol_col)

In [ ]:
df_opt_trade.head()

### large flows
   * calculate the total flow imblance

In [ ]:
all_index = pd.date_range(start=date, end=date+timedelta(days=1), freq='1Min')[:-1]

In [ ]:
def get_total_flow(df_opt_trade_i):
    df_opt_trade_i_f = df_opt_trade_i[['timestamp_dt','amount']]
    return df_opt_trade_i_f.set_index('timestamp_dt').resample('1T', label='right').sum().reindex(all_index).fillna(0)

In [ ]:
df_ret = df_opt_trade.groupby(['type', 'instrument', 'side']).apply(get_total_flow)

df_features = df_ret.unstack('type').unstack('side').unstack('instrument')

flatten_cols = ['_'.join(col).strip() for col in df_features.columns.values]

df_features.columns = flatten_cols

In [ ]:
def trade2flow_features(date):
    exchange = 'deribit'
    data_type= 'trades'
    symbol='OPTIONS'
    df_opt_trade = data_loader(exchange, date, data_type, symbol).pipe(DeribitUtils.parse_time_col).pipe(DeribitUtils.parse_optSymbol_col)
    all_index = pd.date_range(start=date, end=date+timedelta(days=1), freq='1Min')[:-1]
    def get_total_flow(df_opt_trade_i):
        df_opt_trade_i_f = df_opt_trade_i[['timestamp_dt','amount']]
        return df_opt_trade_i_f.set_index('timestamp_dt').resample('1T', label='right').sum().reindex(all_index).fillna(0)
    
    df_ret = df_opt_trade.groupby(['type', 'instrument', 'side']).apply(get_total_flow)
    df_features = df_ret.unstack('type').unstack('side').unstack('instrument')
    flatten_cols = ['_'.join(col).strip() for col in df_features.columns.values]
    df_features.columns = flatten_cols
    return df_features

In [ ]:
rets = []
for date in date_range:
    print(date)
    rets.append(trade2flow_features(date))

In [ ]:
df_flow = pd.concat(rets)

In [ ]:
df_flow.describe()

In [ ]:
df_flow['Call_imb_BTC'] = df_flow['amount_C_buy_BTC'] - df_flow['amount_C_sell_BTC']
df_flow['Put_imb_BTC'] = df_flow['amount_P_sell_BTC'] - df_flow['amount_P_buy_BTC']
df_flow['imb_BTC'] = df_flow['Call_imb_BTC'] - df_flow['Put_imb_BTC']

df_flow['Call_imb_ETH'] = df_flow['amount_C_buy_ETH'] - df_flow['amount_C_sell_ETH']
df_flow['Put_imb_ETH'] = df_flow['amount_P_sell_ETH'] - df_flow['amount_P_buy_ETH']
df_flow['imb_ETH'] = df_flow['Call_imb_ETH'] - df_flow['Put_imb_ETH']


In [ ]:
df_flow.to_pickle(os.path.join(LOCAL_DATA_DIR, 'features','trade_flow.pickle'))

In [ ]:
pd.__version__